In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [5]:
df = pd.read_csv('data/nhanes_2015_2016.csv')
df.SMQ020.value_counts()

2    3406
1    2319
9       8
7       2
Name: SMQ020, dtype: int64

In [7]:
df['SMQ020x'] = df.SMQ020.replace({1:"Yes",2:"No",7:np.nan,9:np.nan})
df.SMQ020x.value_counts()

No     3406
Yes    2319
Name: SMQ020x, dtype: int64

In [28]:
df["RIAGENDRx"] = df.RIAGENDR.replace({1: "Male", 2: "Female"})
ctab = pd.crosstab(index=df['RIAGENDRx'],columns=df['SMQ020x'])

In [26]:
# lets compute the stats for female smokers
# proportion of females that are smokers



0.30484522207267833